In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import matplotlib.pyplot as plt

# sigmoid function
# we don't use the logistic itself since it is slightly more costly
from scipy.special import expit 

import pandas as pd

In [559]:
class MLP():
    def __init__(self, dimensions=[], momentum=0.9, classification=True, 
                 learning_rate=1, use_bias=True, testing=False):
        """A MLP class that implements any amount of fully-connect hidden layers.
        """
        # Check input
        if not dimensions:
            raise ValueError("Must pass input dimensions!")
        if len(dimensions) < 2:
            raise ValueError("At least 2 layers needed")
        
        if testing: # If testing, use always the same seed
            np.random.seed(42)
            
        self.classification = classification
        self.use_bias = use_bias
        self.momentum = momentum
        self.lr = learning_rate
        self.velocities = []
        self.weights = []
        self.grads = []
        
        self.initialize_weights(dimensions)
    
    def initialize_weights(self, dimensions):
        """ Initialize weights - we suposse our bias is built into the weights
        """
        # Add weights and gradients
        for idx, (input_dim, output_dim) in enumerate(dimensions):
            if self.use_bias:
                input_dim += 1
                if idx != len(dimensions) - 1:
                    output_dim += 1
                    
            weight = np.random.uniform(low=-0.1, high=0.100001, 
                                        size=(input_dim, output_dim))
            self.weights.append(weight)
            self.grads.append(np.zeros_like(weight))
            self.velocities.append(np.zeros_like(weight))

    def extend_with_bias(self, matrix):
        # Add bias term to a matrix
        new = np.ones((matrix.shape[0], 
                            matrix.shape[1]+1))
        new[:, :-1] = matrix
        return new
    
    def forward_pass(self, x_train):
        """Performs a forward pass of a dataset through our net
        """
        current = x_train
        outputs = []
        for layer in self.weights:
            output = expit(current.dot(layer))
            outputs.append(output)
            current = output
        
        return outputs

    def backward_pass(self, loss, outputs, input_layers):
        """Performs a backward pass and calculates gradients through our net
        """
        backward_outputs = outputs[::-1]
        idxs = range(len(self.weights), 0,-1)
        last_layer = True
        
        # go through backwards
        for idx, output, prev_layer in zip(idxs, backward_outputs, input_layers[::-1]):
            if last_layer:
                layer_error = loss
                last_layer = False
            else:
                layer_error = np.dot(previous_delta, self.weights[idx].T)
            layer_delta = layer_error * output*(1-output)
            grad = prev_layer.T.dot(layer_delta)

            self.grads[idx-1] = grad
            previous_delta = layer_delta
    
    def fit(self, original_x, original_y, n_iter=100, testing=False,
           verbose=False, mini_batch=32):
        """Trains our model learnable weights with a dataset
        """
        
        if self.use_bias:
            x_train = self.extend_with_bias(original_x)
        else:
            x_train = original_x
            
        y_train = original_y.reshape(len(original_y), 1)
           
        for _ in range(n_iter):   
            for start_idx in range(0, len(x_train), mini_batch):
                x_batch = x_train[start_idx:start_idx+mini_batch]
                y_batch = y_train[start_idx:start_idx+mini_batch]
                
                # Forward pass the mini batch
                outputs = self.forward_pass(x_batch)
                input_layers = [x_batch] + outputs[:-1]

                # Back propagate the loss
                loss = (y_batch - outputs[-1]) / len(x_batch)
                self.backward_pass(loss, outputs, input_layers)

                # Update 
                for idx, (grad, last_v) in enumerate(zip(self.grads, self.velocities)):
                    # Update weights according to momentum
                    new_v = self.momentum * last_v + self.lr*grad
                    self.weights[idx] += new_v
                    # Update the new velocites and set the gradients to 0
                    self.velocities[idx] = new_v
                    self.grads[idx] = np.zeros_like(self.weights[idx])

    def predict(self, x_test):
        if self.use_bias:
            new_x_test = self.extend_with_bias(x_test)
        else:
            new_x_test = x_test
        output = self.forward_pass(new_x_test)[-1]
        
        if self.classification:
            return (output > 0.5).astype(int)
        else:
            return output
    

In [560]:
x = []
y = []

for i in [0, 1]:
    for j in [0, 1]:
        x.append([i, j])
        y.append(i != j)

x = np.array(x)
y = np.array(y)
# Use this if you want more than 1 output node:
#y = np.vstack((y, y)).T
y = y.astype(int)

In [561]:
#x = np.vstack(list(x for i in range(16)))
#y = np.hstack(list(y for i in range(16)))

In [562]:
x.shape, y.shape

((4, 2), (4,))

In [563]:
dimensions = [(2,5), (5,1)]
mlp = MLP(dimensions=dimensions, learning_rate=1, 
         use_bias=True, testing=True)

mlp.fit(x, y, n_iter=2, verbose=False)

In [564]:
dimensions = [(2,5), (5,1)]
mlp = MLP(dimensions=dimensions, learning_rate=1, 
         use_bias=True, testing=True)

mlp.fit(x, y, n_iter=10000, verbose=False)

In [565]:
preds = mlp.predict(x)#.reshape(1, -1)
#pd.DataFrame(np.vstack((preds, y)).T, columns=["MLP predito", "Resposta"])

In [566]:
preds[:4]

array([[0],
       [1],
       [1],
       [0]])